In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
raw_merged_data_path = "final_f1_data.csv"
df_raw = pd.read_csv(raw_merged_data_path)
print("Columns names available in merged csv database:")
print(*list(df_raw.columns), sep='\n')

Columns names avaible in merged csv database:
Year
Race
No
Driver
Car
Pos (race)
Laps (race)
Time/Retired
Points
Pos (qualifying)
Q1
Q2
Q3
Pos (FP1)
Time (FP1)
Gap (FP1)
Laps (FP1)
Pos (FP2)
Time (FP2)
Gap (FP2)
Laps (FP2)
Pos (FP3)
Time (FP3)
Gap (FP3)
Laps (FP3)
Pos (Sprint)
Laps (Sprint)
Time/Retired (Sprint)
Points (Sprint)
Pos (Sprint Quali)
SQ1
SQ2
SQ3
Laps (Sprint Quali)
AvgAirTemp
AvgHumidity
AvgPressure
TotalRainfall
AvgTrackTemp
AvgWindDirection
AvgWindSpeed


### Preprocessing of the combined data used in the models will proceed as follows: 

1. Selecting relevant and useful columns: `Year`, `Race`, `No`, `Driver`, `Car`, `Pos (race)`, `Points`, `AvgAirTemp`, `AvgHumidity`, `AvgPressure`, `TotalRainfall`, `AvgTrackTemp`, `AvgWindDirection`, `AvgWindSpeed`
2. Data Cleaning: Replacing "NC" and "DQ" Values in the `Pos (race)` column with the Last possible position
3. Choosing drivers who participated in each of the years considered.
4. Calculating the driver's power coefficient
5. Calculating the car's power coefficient

In [3]:
# Step 1 
relevant_columns = ["Year", "Race", "No", "Driver", "Car", "Pos (race)", "Points", "AvgAirTemp", "AvgHumidity", "AvgPressure", "TotalRainfall", "AvgTrackTemp", "AvgWindDirection", "AvgWindSpeed"]
df_processed = df_raw.filter(relevant_columns)

In [4]:
# Step 2
df_processed['Pos (race)'] = df_processed['Pos (race)'].astype(str)

# Function to replace "NC" and "DQ" values with the next available place
def replace_nc_dq(group):
    # Sort by "Pos (race)" to determine the next place
    positions = sorted([int(pos) for pos in group['Pos (race)'] if pos.isdigit()])
    next_place = max(positions) + 1 if positions else 1
    # Replace "NC" and "DQ" with the next place
    group['Pos (race)'] = group['Pos (race)'].replace(['NC', 'DQ'], next_place)
    return group

# Apply the function to each race in each year
df_processed = df_processed.groupby(['Year', 'Race'], group_keys=False).apply(replace_nc_dq)

# Convert "Pos (race)" column to numeric, forcing errors to NaN, then fill NaNs with 0 (or any placeholder value if needed)
df_processed['Pos (race)'] = pd.to_numeric(df_processed['Pos (race)'], errors='coerce').fillna(0).astype(int)

In [5]:
# Step 3
# Filter data for each year
drivers_2022 = set(df_processed[df_processed['Year'] == 2022]['Driver'])
drivers_2023 = set(df_processed[df_processed['Year'] == 2023]['Driver'])
drivers_2024 = set(df_processed[df_processed['Year'] == 2024]['Driver'])
# Find common drivers
common_drivers = drivers_2022 & drivers_2023 & drivers_2024
# Filter rows for these common drivers
df_processed = df_processed[df_processed['Driver'].isin(common_drivers)]

In [6]:
# Step 4 and 5
driver_ratings = {
    'Lewis Hamilton': 2.2,
    'Max Verstappen': 2.1,
    'Charles Leclerc': 1.7,
    'Valtteri Bottas': 1.5,
    'Sergio Perez': 1.3,
    'Fernando Alonso': 1.2,
    'Daniel Ricciardo': 1.0,
    'Pierre Gasly': 1.4,
    'Nico Hulkenberg': 1.1,
    'Lando Norris': 1.6,
    'Alexander Albon': 1.2,
    'Lance Stroll': 1.0,
    'Zhou Guanyu': 0.9,
    'Yuki Tsunoda': 1.0,
    'Esteban Ocon': 1.4,
    'Kevin Magnussen': 1.0,
    'George Russell': 1.8,
    'Carlos Sainz': 1.6
}

constructor_ratings = {
    'Mercedes': 1.9,
    'Red Bull': 1.8,
    'Ferrari': 1.6,
    'McLaren': 1.4,
    'Renault': 1.2,
    'AlphaTauri': 1.0,
    'Aston Martin': 0.9,
    'Williams': 0.7,
    'Honda RBPT': 1.7
}

def replace_and_get_max_rating(cell, ratings_dict):
    # Extract relevant names
    found_names = re.findall(r'\b(?:' + '|'.join(re.escape(name) for name in ratings_dict.keys()) + r')\b', cell)
    # Replace with ratings
    ratings = [ratings_dict[name] for name in found_names]
    # Return the maximum rating
    return max(ratings) if ratings else None

df_processed['Car'] = df_processed['Car'].apply(lambda x: replace_and_get_max_rating(x, constructor_ratings))
df_processed['Driver'] = df_processed['Driver'].apply(lambda x: replace_and_get_max_rating(x, driver_ratings))

In [7]:
df_processed

,Year,Race,No,Driver,Car,Pos (race),Points,AvgAirTemp,AvgHumidity,AvgPressure,TotalRainfall,AvgTrackTemp,AvgWindDirection,AvgWindSpeed
0,2022,Bahrain,16,1.7,1.6,1,26.0,23.62,29.49,1010.39,0,28.61,124.43,0.30
1,2022,Bahrain,10,1.4,1.0,20,0.0,23.62,29.49,1010.39,0,28.61,124.43,0.30
2,2022,Bahrain,1,2.1,1.8,19,0.0,23.62,29.49,1010.39,0,28.61,124.43,0.30
3,2022,Bahrain,11,1.3,1.8,18,0.0,23.62,29.49,1010.39,0,28.61,124.43,0.30
4,2022,Bahrain,27,1.1,1.9,17,0.0,23.62,29.49,1010.39,0,28.61,124.43,0.30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1043,2024,Miami,18,1.0,1.9,17,0.0,28.52,59.01,1016.46,0,44.66,152.64,3.08
1044,2024,Miami,23,1.2,1.9,18,0.0,28.52,59.01,1016.46,0,44.66,152.64,3.08
1045,2024,Miami,20,1.0,1.6,19,0.0,28.52,59.01,1016.46,0,44.66,152.64,3.08
1046,2024,Miami,10,1.4,1.2,12,0.0,28.52,59.01,1016.46,0,44.66,152.64,3.08


In [8]:
df_processed.to_csv('data_processing/processed_data.csv', index=False)